# My first Kaggle notebook 

### It is a work in progress and will keep adding more things with time

Hello everyone, in this notebook, I am going to share the linear regression in detail and I shall be using the data of housing prices in King County, USA. This is my first Kaggle notebook and I plan to start with the most basic algorithms of all, regression and the first one of regression technique will be linear regression. I am starting with supervised learning and will keep on adding further notebooks. I read a couple of notebooks on Kaggle which were not that begineer friendly and a lot of assumptions were made about the reader, a lot of basic information was missing. I am writing this notebook so that people can take this as a starting point for working with first dataset and first algorithm-linear regression.

Let us start with the definition first of all, Linear regression is a linear approach to modeling the relationship between a scalar response and one or more explanatory variables. The relationship can be simple with one variable or with many variables, the realtionship between the target variable and explanatory variable will be linear i.e. one degree.

The simplest representation of a linear regression equation is:

y=mx+c

Here, y is the output and x is the input. m is the slope, which is the differential of the equation at any point. Since this is a linear equation, it will stay same throughtout. Intercept is actually the distance of y from the x axis when x=0.

Let us imagine an equation y=2x+1, 2 is the slope and 1 is the intercept. The straight line would somewhat look like this(I will do this after importing all the dependencies):

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import folium
from folium.plugins import HeatMap

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
x = np.linspace(-5,5,10)
y = 2*x+1
plt.plot(x, y, '-r', label='y=2x+1')
plt.title('Graph of y=2x+1')
plt.xlabel('x', color='#1C2833')
plt.ylabel('y', color='#1C2833')
plt.legend(loc='upper left')
plt.grid()
plt.show()

In the graph above, you see a straight line. This is a simple linear regression problem. Being linear means straight line or having a degree of one in the algebraic equation. We can also make a linear equation with more than one input variable. It is called multiple linear regression. It is of the form:

y=m1x1 + m2x2 + m3x3 + m4x4 +........

Now we will start with the problem at hand, which is about predicting house price in King county, USA. We have a number of Input variables and our output should be the price. So, essentially, we have to use machine learning algorithm to predict the price of houses in King County, USA using the various features like bedrooms, bathrooms, sqft living area space, etc. We will start off by importing the dataset and looking at the first five rows of the data.

In [ ]:
df=pd.read_csv("/kaggle/input/housesalesprediction/kc_house_data.csv")
df.head(5)

In [ ]:
df.columns

In [ ]:
df['view'].value_counts()

## Exploring the dataset

I am going to perform very basic EDA here. You can add more interactive and informative graphs to display your data better. In the columns above, we will decide which columns to keep and which to drop or modify. Let us first try to look at the columns yr_built and yr_renovated and try to modify them. 

Since the main goal of this notebook is to construct a working model which has the capability of predicting the price of houses, we will need to separate the dataset into features and the target variable. The target variable, 'price', will be the variable we seek to predict and other variables are features  These are stored in features and prices, respectively. 

The first step would be look at the property of our target variable

In [ ]:
print("The maximum price of a house is {}".format(df['price'].max()))
print("The minimum price of a house is {}".format(df['price'].min()))
print("The mean price of houses is {}".format(df['price'].mean()))
print("The median price of houses is {}".format(df['price'].median()))

In [ ]:
print(df['yr_built'].max())
print(df['yr_built'].min())

The newest house was built in 2015 so we can calculate the age of houses from 2015 by subtracting the houses built from 2015. Let us explore the next feature, year renovted.

In [ ]:
df['age']=2015-df['yr_built']

In [ ]:
print(df['yr_renovated'].value_counts())
print(df['yr_renovated'].max())
print(df['yr_renovated'].min())

In [ ]:
df['renovation']=2015-df['yr_renovated']

In the dataset above, we do not need the columns id and price, so we will drop these in the dataframe. We have modified the yr_built and yr_renovated so we will drop both as well. In the subsequent lines, we will have a look at the statistical parameters of our data and then look out for missing values, if there are any. 

In [ ]:
df.drop(columns=["id",'date','yr_built','yr_renovated'], inplace=True)
df.describe()

In [ ]:
df.isna().sum()

We can see there are no missing values in the data and now we will find correlation of various variables with one another. We will use pearson correlation coefficient. Correlation is a quantitative assessment that measures both the direction and the strength of this tendency to vary together.A number closer to 1 indicates strong correlation while a number of closer to 0 indicates weak correlation. A positive coefficient means proportional realtion, i.e.the variable 2 moves in the same direction as variable 1, with which we are comparing.

Apart from seeing it numerically, we can also see it visually with the help of matplotlib and seaborn libraries in Python.

In [ ]:
df.corr()

In [ ]:
def correlation_heatmap(df1):
    _, ax = plt.subplots(figsize = (15, 10))
    colormap= sns.diverging_palette(220, 10, as_cmap = True)
    sns.heatmap(df.corr(), annot=True, cmap = colormap)

correlation_heatmap(df)  

Now, we have three types of variables here:

**1. Continuous:** sqft_living, sqft_lot, age, renovation

**2. Categorical:** Bedrooms, Bathrooms, Waterfront, view, condition, grade, floors

**3. Nominal:** Zip code, lat, long

For continuous data, we will plot scatterplot and for categorical data, we will prefer a box plot. The nominal data is mainly geographical in nature, so we will try to plot maps and see if there is any pattern!

### Visualizing Continuous data on pairplot

In [ ]:
g = sns.pairplot(df[['sqft_above','price','sqft_living','bedrooms', 'age']], 
                 hue='bedrooms', palette='husl',height=6,)

### Box plots for categorical data:

In [ ]:
sns.set(style="whitegrid", font_scale=1)
f, axes = plt.subplots(1, 2,figsize=(18,5))
sns.boxplot(x=df['bedrooms'],y=df['price'], ax=axes[0])
sns.boxplot(x=df['floors'],y=df['price'], ax=axes[1])
sns.despine(left=True, bottom=True)
axes[0].set(xlabel='Bedrooms', ylabel='Price')
axes[0].yaxis.tick_left()
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='Floors', ylabel='Price')

f, axe = plt.subplots(1, 1,figsize=(18,5))
sns.despine(left=True, bottom=True)
sns.boxplot(x=df['bathrooms'],y=df['price'], ax=axe)
axe.yaxis.tick_left()
axe.set(xlabel='Bathrooms', ylabel='Price')

f, axes = plt.subplots(1, 2,figsize=(18,5))
sns.boxplot(x=df['waterfront'],y=df['price'], ax=axes[0])
sns.boxplot(x=df['view'],y=df['price'], ax=axes[1])
sns.despine(left=True, bottom=True)
axes[0].set(xlabel='waterfront', ylabel='Price')
axes[0].yaxis.tick_left()
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='view', ylabel='Price')

f, axes = plt.subplots(1, 2,figsize=(18,5))
sns.boxplot(x=df['condition'],y=df['price'], ax=axes[0])
sns.boxplot(x=df['grade'],y=df['price'], ax=axes[1])
sns.despine(left=True, bottom=True)
axes[0].set(xlabel='condition', ylabel='Price')
axes[0].yaxis.tick_left()
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='grade', ylabel='Price')

### Key takeaways from the boxplots:

1. The house of houses with more number of bathrooms are higher but it kind of plateaus near 7-8 bathrooms
2. The price of houses increases for houses with 0-2.5 (around 3) floors and then subsequently decreases
3. Houses with more number of floors have higher price.
4. Houses having a waterfront are valued higher 
5. I have no idea about what numbers indicate in case of view and grade but prices increase as their number increase 
6. Condition does not convey much information. 

{Note: Have to add visualization for the zipcode, latitudes and longitudes}

Codes for making these visualization can be found in the documentation of respective libraries. 

Now, there are two options for us, we can either one variable to predict prices or we can use a combination of features to predict the target variable, which is the price of our houses in King county, USA. At first, we will choose one parameter and train our model based on that.

As mentioned before, in our dataset, price is the dependent or target variable which is the target of our model. Other variables than price will be independent variables which will help us determine the target variable. So we will first make a Simple linear regression model which will use the dependent variable and one independent variable to train and test our model.

# Simple linear regression model

LinearRegression fits a linear model with coefficients w= (w1,w2,w3,...) to minimize the residual sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation. Linear regression is essentially a model based on Ordinary Least sqaure algorithm. 

## Training and testing the model

In a machine learning program, we train the model with some dataset and then test the model with some other dataset. It is similar to how we teach kids to learn. We give them textbooks with the entire syllabus to train themselves and then we test them from some questions from the textbook. We evaluate the kids based on how many answers they get correct. Similarly, we have some parameters to check whether our model is able to get the right answers or not. 

We will have to split the data in two halves as training and testing data. Training data will be used to train and the other to test. We will use the data from same dataset to avoid variations. We will keep 70% of the dataset for training and we will split using scikit learn's train_test_split. We will fit the model using linear regression module. 

## Evaluating our Machine Learning model 

There are a number of metrics which can be used to check the performance of our regression model. Depeding upon the problem and the data, suitable metric can be used.

I am using the content from: https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b

**1. R squared or adjusted R squared:** R Square measures how much of variability in dependent variable can be explained by the model. It is square of Correlation Coefficient(R) and that is why it is called R Square. R Square is calculated by the sum of squared of prediction error divided by the total sum of square which replace the calculated prediction with mean. R Square value is between 0 to 1 and bigger value indicates a better fit between prediction and actual value.
R Square is a good measure to determine how well the model fits the dependent variables. 

However, it does not take into consideration of overfitting problem. If your regression model has many independent variables, because the model is too complicated, it may fit very well to the training data but performs badly for testing data. That is why Adjusted R Square is introduced because it will penalise additional independent variables added to the model and adjust the metric to prevent overfitting issue.


**2. Mean Square Error(MSE)/Root Mean Square Error(RMSE):** While R Square is a relative measure of how well the model fits dependent variables, Mean Square Error is an absolute measure of the goodness for the fit. MSE is calculated by the sum of square of prediction error which is real output minus predicted output and then divide by the number of data points. It gives you an absolute number on how much your predicted results deviate from the actual number. 

Root Mean Square Error(RMSE) is the square root of MSE. It is used more commonly than MSE because firstly sometimes MSE value can be too big to compare easily. Secondly, MSE is calculated by the square of error, and thus square root brings it back to the same level of prediction error and make it easier for interpretation.

**3. Mean Absolute Error(MAE):** Mean Absolute Error(MAE) is similar to Mean Square Error(MSE). However, instead of the sum of square of error in MSE, MAE is taking the sum of absolute value of error. Compared to MSE or RMSE, MAE is a more direct representation of sum of error terms. MSE gives larger penalisation to big prediction error by square it while MAE treats all errors the same.


We will start off by creating a function which outputs all the evaluation metrics, so that it will be easier to test every model without writing a lot of codes! 



Our dependent variable is fixed, we have to choose the dependent variable. So, we will make use of the correlation matrix to find out which should the independent variable we should use. If we look at the matrix, price is highly correlated to the 'sqft_living' so we will start with this variable. At first, we will visualize the relationship between both variables and determine if its linear or not

In [ ]:
f, ax = plt.subplots(figsize=(16, 12))
sns.regplot(x="price", y="sqft_lot", data=df)

From the plot, it is clear that they both share a linear relationship with some outliers. Let us have a look at the top 100 elements and how it looks like:

In [ ]:
f, ax = plt.subplots(figsize=(16, 12))
sns.regplot(x="price", y="sqft_lot", data=df.iloc[0:100,:])

As expected, the relationship is linear. It is time to start building our model and we have chosen simple linear regression. Let us try to fit a line on our dataset and see what is the value of m and c in the straight line that will fit. The ordinary least square method will be used to fit the line. Ordinary least squares, or linear least squares, estimates the parameters in a regression model by minimizing the sum of the squared residuals. This method draws a line through the data points that minimizes the sum of the squared differences between the observed values and the corresponding fitted values.

We will use the python libraries scikit learn and statsmodels to apply Linear regression.

In [ ]:
feature_cols = ['sqft_living']
X = df[feature_cols]
y = df.price

reg = LinearRegression()
hp=reg.fit(X, y)

print(hp.intercept_)
print(hp.coef_)

The line fitting the above coeeficients will be y= 280x - 43580 approximately. There is always an error involved as we cannot find a perfect fit to the data we have at hand.

R-squared is a statistical measure of how close the data are to the fitted regression line. It is also known as the coefficient of determination, or the coefficient of multiple determination for multiple regression. The definition of R-squared is fairly straight-forward; it is the percentage of the response variable variation that is explained by a linear model. Or:

R-squared = Explained variation / Total variation

R-squared is always between 0 and 100%:

0% indicates that the model explains none of the variability of the response data around its mean. 100% indicates that the model explains all the variability of the response data around its mean.

In [ ]:
hp.score(X, y)

The value is low, we can improve the model by including more features and instead of a simple linear regression, we will use multiple linear regression. Furthermore, one more thing to note here is, with r sqaured is we cannot be so sure whether it is the correct measure to evaluate the performance of our model.

It's hard to say. The threshold for a good R-squared value depends widely on the domain. Therefore, it's most useful as a tool for comparing different models.

# Multiple linear regression

We have tried simple linear regression on our model and now we will try multiple linear regression using a combination of factors and we will see what works best. We will have to perform feature selection:

Feature selection
We will have to look for two correlations:

1. The correlation between dependent and independent variables: We will use features which are highly correlated to the dependent variable.

2. Correlation between independent variables: We will remove one of the two highly correlated features as they will essentially behave in the same manner and using a lot of features might result in overfitting and an increase in training and evaluating model.

We will be trying a combination of various features to see which gives us a better r sqaured values. {Note: Have to update with adjusted r squared value}

In [ ]:
feature_cols1 = ['sqft_living15', 'sqft_above', 'grade', 'sqft_living', 'bathrooms']
X = df[feature_cols1]
y = df.price

hp1 = LinearRegression()
hp1.fit(X, y)

print(hp1.intercept_)
print(hp1.coef_)
hp1.score(X, y)

In [ ]:
feature_cols2 = ['sqft_living15', 'sqft_above', 'grade', 'sqft_living', 'bathrooms','lat', 'sqft_basement', 'view' , 'waterfront', 'floors', 'bedrooms']
X = df[feature_cols2]
y = df.price

hp1 = LinearRegression()
hp1.fit(X, y)


print(hp1.intercept_)
print(hp1.coef_)
hp1.score(X, y)

In [ ]:
feature_cols3 = ['sqft_living15', 'sqft_above', 'grade', 'sqft_living', 'bathrooms','lat', 'sqft_basement', 'view' , 'waterfront', 'floors', 'bedrooms', 'age']
X = df[feature_cols3]
y = df.price


hp1 = LinearRegression()
hp1.fit(X, y)

print(hp1.intercept_)
print(hp1.coef_)
hp1.score(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(df,train_size = 0.7,random_state=0)

reg = linear_model.LinearRegression()

X_train = np.array(train_data['sqft_living'], dtype=pd.Series).reshape(-1, 1)
Y_train = np.array(train_data['price'], dtype=pd.Series).reshape(-1,1)
X_test=np.array(test_data['sqft_living'], dtype=pd.Series).reshape(-1, 1)
Y_test=np.array(test_data['price'], dtype=pd.Series).reshape(-1, 1)

reg.fit(X_train,Y_train)

Now we have fit our data to the Linear regression model. We will find the coefficient and intercept for the linear relationship to be subsituted in the linear equation: 

y=mx+c

In [ ]:
LinearRegression()
print(reg.coef_,reg.intercept_)